<a href="https://colab.research.google.com/github/htapia/damju/blob/master/DAMJu_S2_L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semana 2 Libreta 2

## Establecer conexion con el dispositivo

In [21]:
!adb devices

List of devices attached
98f2eeee	device



In [23]:
%%bash
adb kill-server
# adb tcpip 5555
adb devices

List of devices attached
* daemon not running. starting it now on port 5037 *
* daemon started successfully *
98f2eeee	device



connected to 192.168.1.75:5554


In [7]:
# use this if you are connecting via usb (PRIVATE SL4A Server)
!adb forward tcp:9999 tcp:2222

error: device '(null)' not found
error: device '(null)' not found


In [13]:
# use this if you connect via wifi (PUBLIC SL4A Server)
# !adb tcpip 5554
!adb connect 192.168.1.75:5554 # < usar su propia IP >

unable to connect to 192.168.17.73:5555


## Primera aplicacion

Con el sistema de desarrollo listo, vamos a crear una primera applicacion de forma interactiva. Al final podemos desplegarla en el dispositivo y ejecutarla ahi. 

Esta libreta tiene como objetivo crear una aplicacion que ponga el dispositivo en modo sielncio si se coloca con la pantalla hacia abajo sobre una superficie plana

Para lograrlo realizaremos los siguietnes pasos:

1. Determinar si el dispositivo se encuentra orientado con la pantalla haciar arriba
2. Determinar si el dispositivo esta en posicion horizontal en reposo o moviendose

In [0]:
import androidhelper, time

In [0]:
droid = androidhelper.Android(['192.168.1.76','2222'])

In [45]:
# Hola mundo
droid.makeToast('Hola, mundo!')

Result(id=0, result=None, error=None)

In [0]:
line = droid.dialogGetInput()

In [35]:
s = "Hola, %s" % (line.result,)
print s

Hola, un valor


In [33]:
droid.makeToast(s)

Result(id=2, result=None, error=None)

## Explorando la API de Python en Android

Ahora vamos a crear codigo que invoca diferentes componentes de la API en android.

In [0]:
# comenzamos por invocar algunas librearias adicionales
import sys
import time
import types
# import androidhelper  # SOLO EN EL CASO QUE NO LA HAYA INVOCADO ANTERIORMENT

In [0]:
def test_clipboard():
  previous = droid.getClipboard().result
  msg = 'Hola, clase!'
  droid.setClipboard(msg)
  echo = droid.getClipboard().result
  droid.setClipboard(previous)
  return echo == msg

In [0]:
def test_speak():
  result = droid.ttsSpeak('Hola, clase')
  return result.error is None

In [0]:
def test_make_toast():
  result = droid.makeToast('Hello, world!')
  return result.error is None

In [0]:
def test_notify():
  result = droid.notify('Test Title', 'Hello, world!')
  return result.error is None

In [0]:
def test_vibrate():
  result = droid.vibrate()
  return result.error is None

In [0]:
def test_alert_dialog():
  title = 'User Interface'
  message = 'Welcome to the SL4A integration test.'
  droid.dialogCreateAlert(title, message)
  droid.dialogSetPositiveButtonText('Continue')
  droid.dialogShow()
  response = droid.dialogGetResponse().result
  return response['which'] == 'positive'

In [0]:
def test_alert_dialog_with_buttons():
  title = 'Alert'
  message = ('This alert box has 3 buttons and '
             'will wait for you to press one.')
  droid.dialogCreateAlert(title, message)
  droid.dialogSetPositiveButtonText('Yes')
  droid.dialogSetNegativeButtonText('No')
  droid.dialogSetNeutralButtonText('Cancel')
  droid.dialogShow()
  response = droid.dialogGetResponse().result
  return response['which'] in ('positive', 'negative', 'neutral')

In [0]:
def test_spinner_progress():
  title = 'Spinner'
  message = 'This is simple spinner progress.'
  droid.dialogCreateSpinnerProgress(title, message)
  droid.dialogShow()
  time.sleep(2)
  droid.dialogDismiss()
  return True

In [0]:
def test_horizontal_progress():
  title = 'Horizontal'
  message = 'This is simple horizontal progress.'
  droid.dialogCreateHorizontalProgress(title, message, 50)
  droid.dialogShow()
  for x in range(0, 50):
    time.sleep(0.1)
    droid.dialogSetCurrentProgress(x)
  droid.dialogDismiss()
  return True

In [0]:
def test_alert_dialog_with_list():
  title = 'Alert'
  droid.dialogCreateAlert(title)
  droid.dialogSetItems(['foo', 'bar', 'baz'])
  droid.dialogShow()
  response = droid.dialogGetResponse().result
  return True


In [0]:
def test_alert_dialog_with_single_choice_list():
  title = 'Alert'
  droid.dialogCreateAlert(title)
  droid.dialogSetSingleChoiceItems(['foo', 'bar', 'baz'])
  droid.dialogSetPositiveButtonText('Yay!')
  droid.dialogShow()
  response = droid.dialogGetResponse().result
  return True

In [0]:
def test_alert_dialog_with_multi_choice_list():
  title = 'Alert'
  droid.dialogCreateAlert(title)
  droid.dialogSetMultiChoiceItems(['foo', 'bar', 'baz'], [])
  droid.dialogSetPositiveButtonText('Yay!')
  droid.dialogShow()
  response = droid.dialogGetResponse().result
  return True

## Pruebas 2

In [0]:
def test_imports():
  try:
    import termios
    import bs4 as BeautifulSoup
    import pyxmpp2 as xmpp
    from xml.dom import minidom
  except ImportError:
    return False
  return True

In [0]:
def test_get_running_packages():
  result = droid.getRunningPackages()
  return result.error is None

In [0]:
def event_loop():
  for i in range(10):
    time.sleep(1)
    droid.eventClearBuffer()
    time.sleep(1)
    e = droid.eventPoll(1)
    if e.result is not None:
      return True
  return False    

In [0]:
def test_battery():
  droid.batteryStartMonitoring()
  time.sleep(1)
  try:
    return bool(droid.batteryGetStatus())
  finally:
    droid.batteryStopMonitoring()

In [0]:
def test_sensor():
  # acelerometro 1 lectura por segundo
  droid.startSensingTimed(2, 1000)
  try:
    return event_loop()
  finally:
    droid.stopSensing()

In [0]:
def test_gps():
  droid.startLocating()
  try:
    return event_loop()
  finally:
    droid.stopLocating()

In [0]:
def test_get_last_known_location():
  result = droid.getLastKnownLocation()
  return result.error is None

In [0]:
def test_geocode():
  result = droid.geocode(0.0, 0.0, 1)
  return result.error is None

In [0]:
def test_phone_state():
  droid.startTrackingPhoneState()
  try:
    return event_loop()
  finally:
    droid.stopTrackingPhoneState()

In [0]:
def test_ringer_silent():
  result1 = droid.toggleRingerSilentMode()
  result2 = droid.toggleRingerSilentMode()
  return result1.error is None and result2.error is None

In [0]:
def test_ringer_volume():
  get_result = droid.getRingerVolume()
  if get_result.error is not None:
    return False
  droid.setRingerVolume(0)
  set_result = droid.setRingerVolume(get_result.result)
  if set_result.error is not None:
    return False
  return True

In [0]:
def test_wifi():
  result1 = droid.toggleWifiState()
  result2 = droid.toggleWifiState()
  droid.toggleWifiState(True)  # Make sure wifi ends up ON, as it interferes with other tests
  return result1.error is None and result2.error is None

# Fin de sesion.